# Data Extraction

In [180]:
import io
import os
import re
import tarfile
from pathlib import Path

import chardet
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession

In [181]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

In [182]:
APACHE_SPAM_ASSASSIN = "https://spamassassin.apache.org/old/publiccorpus"
DATA_FOLDER = '../data'

In [4]:
def pull_data():
    response = requests.get(APACHE_SPAM_ASSASSIN)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    file_paths = [link.get('href') for link in soup.find_all('a')]
    file_paths = [path for path in file_paths if path.split('.')[-1] == 'bz2']

    available_data = set(os.listdir(DATA_FOLDER))
    file_paths = [path for path in file_paths if path not in available_data and path != 'corpus.parquet']

    if len(file_paths) == 0:
        print('No data to pull')

    for file_path in file_paths:
        print(f'Pulling {file_path}')
        response = requests.get(f"{APACHE_SPAM_ASSASSIN}/{file_path}")
        response.raise_for_status()

        file_object = io.BytesIO(response.content)
        tar = tarfile.open(fileobj=file_object, mode="r:bz2")

        extract_dir = Path(DATA_FOLDER)
        extract_path = extract_dir.joinpath(Path(file_path))

        extract_dir.mkdir(exist_ok=True)
        tar.extractall(extract_path)

        tar.close()

    size = get_directory_size("../data")
    print(f"Data directory size: {size} bytes")


def get_directory_size(directory):
    total = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total += os.path.getsize(fp)

    return total

In [10]:
pull_data()

No data to pull
Data directory size: 121604826 bytes


In [185]:
def parse_data():
    successful_files = []
    failed_files = []
    corpus_df = pd.DataFrame(columns=['date', 'difficulty', 'category', 'collection', 'body'])
    for dirpath, dirnames, filenames in os.walk(DATA_FOLDER):
        print(dirpath)
        for filename in filenames:
            corpus_path = os.path.join(dirpath, filename)
            corpus_df, successful_files, failed_files = incorporate(corpus_df, corpus_path, dirpath, successful_files,
                                                                    failed_files)
    return corpus_df, successful_files, failed_files


def incorporate(df, corpus_path, dirpath, successful_files, failed_files):
    rawdata = open(corpus_path, 'rb').read()
    result = chardet.detect(rawdata)
    encoding = result['encoding']

    with open(corpus_path, 'r', encoding=encoding) as f:
        [(date, *ids)] = re.findall(r'([\d]{8})_([a-z]*)_?([a-z]*)_?([\d]*)\.tar\.bz2.*',
                                    dirpath)
        [difficulty, category, collection] = parse_ids(ids)
        try:
            body = f.read()
            row = pd.DataFrame({'date': date,
                                'difficulty': difficulty,
                                'category': category,
                                'collection': collection,
                                'body': body}, index=[0, 1, 2, 3, 4])
            df = pd.concat([df, row], ignore_index=True)
            successful_files.append(corpus_path)
        except Exception as E:
            print(E)
            failed_files.append(corpus_path)
    return df, successful_files, failed_files


def parse_ids(ids):
    cls, difficulty, collection = None, None, None
    match ids:
        case [cls, '', '']:
            difficulty = None
            collection = None
        case [difficulty, cls, '']:
            collection = None
        case [difficulty, cls, collection]:
            pass
        case _:
            pass

    return [difficulty, cls, collection]


def save_as_parquet(df):
    spark_df = spark.createDataFrame(df)
    spark_df.write.parquet(f'{DATA_FOLDER}/corpus.parquet')

In [187]:
if 'corpus.parquet' in set(os.listdir(DATA_FOLDER)):
    df = spark.read.parquet(f'{DATA_FOLDER}/corpus.parquet')
    corpus_df = df.toPandas()
else:
    corpus_df, successful_files, failed_files = parse_data()
    print(failed_files)
    save_as_parquet(corpus_df)

../data
../data/20030228_hard_ham.tar.bz2
../data/20030228_hard_ham.tar.bz2/hard_ham
../data/20021010_hard_ham.tar.bz2
../data/20021010_hard_ham.tar.bz2/hard_ham
../data/20030228_spam_2.tar.bz2
../data/20030228_spam_2.tar.bz2/spam_2
'cp949' codec can't decode byte 0xb7 in position 4589: illegal multibyte sequence
'charmap' codec can't decode byte 0x90 in position 2832: character maps to <undefined>
'charmap' codec can't decode byte 0x9e in position 3019: character maps to <undefined>
'big5' codec can't decode byte 0xf9 in position 2730: illegal multibyte sequence
../data/20030228_easy_ham_2.tar.bz2
../data/20030228_easy_ham_2.tar.bz2/easy_ham_2
'cp949' codec can't decode byte 0xb7 in position 3140: illegal multibyte sequence
../data/20030228_spam.tar.bz2
../data/20030228_spam.tar.bz2/spam
'gb2312' codec can't decode byte 0xeb in position 1587: illegal multibyte sequence
'cp949' codec can't decode byte 0xb7 in position 3073: illegal multibyte sequence
'cp949' codec can't decode byte 0xb

23/06/18 18:09:30 WARN TaskSetManager: Stage 2 contains a task of very large size (11918 KiB). The maximum recommended task size is 1000 KiB.
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/06/18 18:09:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054

In [188]:
corpus_df

,date,difficulty,category,collection,body
0,20030228,hard,ham,None,Return-Path: <bounce-lghtml-2534368@sprocket.l...
1,20030228,hard,ham,None,Return-Path: <bounce-lghtml-2534368@sprocket.l...
2,20030228,hard,ham,None,Return-Path: <bounce-lghtml-2534368@sprocket.l...
3,20030228,hard,ham,None,Return-Path: <bounce-lghtml-2534368@sprocket.l...
4,20030228,hard,ham,None,Return-Path: <bounce-lghtml-2534368@sprocket.l...
...,...,...,...,...,...
53650,20050311,spam,,2,Received: from actioncouriers.com ([207.225.37...
53651,20050311,spam,,2,Received: from actioncouriers.com ([207.225.37...
53652,20050311,spam,,2,Received: from actioncouriers.com ([207.225.37...
53653,20050311,spam,,2,Received: from actioncouriers.com ([207.225.37...


# Preprocessing

In [193]:
def extract_headers(row):
    row['has_headers'] = True
    try:
        [headers, body] = row['body'].split('\n\n', maxsplit=1)
        header_rows = headers.split('\n')
        current_key = None
        for h in header_rows:
            row, current_key = parse_header(row, h, current_key)
        row['body'] = body
    except Exception as e:
        row['header_error'] = e
        row['has_headers'] = False
    return row


def parse_header(row, h, current_key):
    match re.split(r"([\w\-]+)\:\s*", h):
        case ['', key, value]:
            row[key.lower()] = value
            current_key = key.lower()
        case [value]:
            if current_key:
                row[current_key] += value
        case _:
            pass
    return row, current_key

In [194]:
preproc_df = corpus_df.drop_duplicates(subset='body')
preproc_df = preproc_df.apply(extract_headers, axis=1)

In [195]:
reduced_preproc_df = preproc_df.dropna(thresh=1000, axis=1)

In [197]:
def extract_email_address(row, cols):
    for col in cols:
        if str(row[col]) and type(row[col]) != list:
            row[col] = re.findall(r"([^\s\<]+\@[^\s\>]+)", str(row[col])) or None
    return row

In [198]:
envelope_cols = ['delivered-to', 'errors-to', 'from', 'in-reply-to', 'list-id', 'message-id', 'received', 'references',
                 'reply-to', 'return-path', 'sender', 'to', 'x-beenthere']
extracted_emails_df = reduced_preproc_df.apply(lambda r: extract_email_address(r, envelope_cols), axis=1)

In [199]:
matching_cols = [
    ['delivered-to', 'to'],
    ['errors-to', 'from'],
    ['errors-to', 'return-path'],
    ['from', 'reply-to'],
    ['from', 'return-path'],
    ['from', 'sender'],
    ['x-beenthere', 'list-id']
]


def envelope_cols_match(row):
    for [col1, col2] in matching_cols:
        values_exist = row[col1] and row[col2]
        if not values_exist:
            return row
        row[f'feat-match-{col1}-{col2}'] = 1 if set(row[col1]) == set(row[col2]) else 0
    return row

In [200]:
matched_envelopes_df = extracted_emails_df.apply(envelope_cols_match, axis=1)

In [201]:
def get_feat_cols(df):
    return [col for col in df.columns if re.match(r"^feat\-.*$", col) is not None]

In [202]:
matched_envelopes_df[get_feat_cols(matched_envelopes_df)].describe()

,feat-match-delivered-to-to,feat-match-errors-to-from,feat-match-errors-to-return-path,feat-match-from-reply-to,feat-match-from-return-path,feat-match-from-sender
count,7760.000000,4569.000000,4568.000000,1662.000000,1662.000000,1654.000000
mean,0.147938,0.003502,0.992557,0.293020,0.010229,0.007860
std,0.355062,0.059079,0.085961,0.455285,0.100648,0.088333
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [203]:
message_ids = {}


def duplicate_message_id(row):
    if not row['message-id']:
        matched_envelopes_df.loc[row.name, 'feat-duplicate-message-id'] = 0
        return row

    for message_id in row['message-id']:
        if message_id not in message_ids:
            message_ids[message_id] = row.name
            matched_envelopes_df.loc[row.name, 'feat-duplicate-message-id'] = 0
        else:
            matched_envelopes_df.loc[message_ids[message_id], 'feat-duplicate-message-id'] = 1
    return row


matched_envelopes_df.apply(duplicate_message_id, axis=1)
duplicate_messages_df = matched_envelopes_df

In [204]:
def prepare_for_encoding(row, col):
    if pd.isna(row[col]):
        return row

    row[col] = row[col].lower()
    row[col] = row[col].split(' ')[0]
    return row


def one_hot_encode(df, col):
    cte_df = df.apply(lambda r: prepare_for_encoding(r, col), axis=1)
    encoded_cte_df = pd.get_dummies(cte_df[col], prefix=f'feat-encoded-{col}', dtype=int)
    return pd.concat([cte_df, encoded_cte_df], axis=1)


with_encoded_cte_df = one_hot_encode(duplicate_messages_df, 'content-transfer-encoding')

In [205]:
with_encoded_precedence_df = one_hot_encode(with_encoded_cte_df, 'precedence')
with_encoded_precedence_df[get_feat_cols(with_encoded_precedence_df)]

,feat-match-delivered-to-to,feat-match-errors-to-from,feat-match-errors-to-return-path,feat-match-from-reply-to,feat-match-from-return-path,feat-match-from-sender,feat-duplicate-message-id,feat-encoded-content-transfer-encoding_7bit,feat-encoded-content-transfer-encoding_8bit,feat-encoded-content-transfer-encoding_base64,feat-encoded-content-transfer-encoding_binary,feat-encoded-content-transfer-encoding_quoted-printable,feat-encoded-precedence_bulk,feat-encoded-precedence_first-class,feat-encoded-precedence_list,feat-encoded-precedence_normal
0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0
5,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,1,0,0,0
10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0,0
15,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1,0,0,0,0
20,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,1,0,0,0,1,0,0,0
46665,0.0,0.0,1.0,NaN,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0
46675,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,0,0
46680,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,0,0


In [206]:
with_encoded_mailman_version_df = one_hot_encode(with_encoded_precedence_df, 'x-mailman-version')
with_encoded_mailman_version_df[get_feat_cols(with_encoded_mailman_version_df)]

,feat-match-delivered-to-to,feat-match-errors-to-from,feat-match-errors-to-return-path,feat-match-from-reply-to,feat-match-from-return-path,feat-match-from-sender,feat-duplicate-message-id,feat-encoded-content-transfer-encoding_7bit,feat-encoded-content-transfer-encoding_8bit,feat-encoded-content-transfer-encoding_base64,...,feat-encoded-x-mailman-version_2.0.1,feat-encoded-x-mailman-version_2.0.10,feat-encoded-x-mailman-version_2.0.11,feat-encoded-x-mailman-version_2.0.3,feat-encoded-x-mailman-version_2.0.6,feat-encoded-x-mailman-version_2.0.7,feat-encoded-x-mailman-version_2.0.8,feat-encoded-x-mailman-version_2.0.9-sf.net,feat-encoded-x-mailman-version_2.1b5,feat-encoded-x-mailman-version_2.1b5+
0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
15,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,1,0,...,0,0,1,0,0,0,0,0,0,0
46665,0.0,0.0,1.0,NaN,NaN,NaN,NaN,1,0,0,...,0,0,1,0,0,0,0,0,0,0
46675,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0
46680,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [207]:
def parse_msmail_priority(row):
    if pd.isna(row['x-msmail-priority']):
        return row

    match row['x-msmail-priority']:
        case 'normal':
            row['feat-x-msmail-priority'] = 1
        case 'medium':
            row['feat-x-msmail-priority'] = 1
        case 'low':
            row['feat-x-msmail-priority'] = 2
        case 'high':
            row['feat-x-msmail-priority'] = 3
        case _:
            row['feat-x-msmail-priority'] = -1

    return row


with_msmail_priority_df = with_encoded_mailman_version_df.apply(lambda r: prepare_for_encoding(r, 'x-msmail-priority'),
                                                                axis=1)
with_categorical_msmail_priority_df = with_msmail_priority_df.apply(parse_msmail_priority, axis=1)
with_categorical_msmail_priority_df['feat-x-msmail-priority'].describe()

count    1149.000000
mean        1.075718
std         0.387537
min        -1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: feat-x-msmail-priority, dtype: float64

In [208]:
def parse_priority(row):
    if pd.isna(row['x-priority']):
        return row

    priority = row['x-priority']
    if re.findall(r"Highest", priority):
        row['feat-x-priority'] = 0
    else:
        row['feat-x-priority'] = int(re.search(r'\d', priority).group())
    return row


with_categorical_priority_df = with_categorical_msmail_priority_df.apply(parse_priority, axis=1)
with_categorical_priority_df['feat-x-priority'].describe()

count    1488.000000
mean        2.828629
std         0.588674
min         0.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         5.000000
Name: feat-x-priority, dtype: float64

In [209]:
def parse_spam_status(row):
    spam_status = row['x-spam-status']
    if pd.isna(spam_status):
        return row

    match spam_status.split(', ')[0].lower():
        case 'no':
            row['feat-x-spam-status'] = 0
        case 'yes':
            row['feat-x-spam-status'] = 1
        case _:
            row['feat-x-spam-status'] = -1
    return row


with_binary_spam_status_df = with_categorical_priority_df.apply(parse_spam_status, axis=1)
with_binary_spam_status_df['feat-x-spam-status'].describe()

count    2155.000000
mean        0.007425
std         0.085866
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: feat-x-spam-status, dtype: float64

In [210]:
def parse_spam_level(row):
    spam_level = row['x-spam-level']
    if pd.isna(spam_level):
        return row

    row['feat-x-spam-level'] = len(spam_level.rstrip().lstrip())
    return row


with_categorical_spam_level_df = with_binary_spam_status_df.apply(parse_spam_level, axis=1)
with_categorical_spam_level_df['feat-x-spam-level'].describe()

count    2156.000000
mean        0.067718
std         0.628818
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.000000
Name: feat-x-spam-level, dtype: float64

In [211]:
embedding_cols = ['cc', 'content-type', 'subject', 'x-mailer']

In [212]:
def get_cc_count(row):
    if pd.isna(row['cc']):
        return row

    row['feat-cc-count'] = len(row['cc'].split(','))
    return row


with_cc_count_df = with_categorical_spam_level_df.apply(get_cc_count, axis=1)
with_cc_count_df['feat-cc-count'].describe()

count    1962.000000
mean        3.007136
std         6.011204
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        73.000000
Name: feat-cc-count, dtype: float64

In [213]:
ct_subfields = set()


def parse_content_type(row):
    content_type = row['content-type']
    if pd.isna(content_type):
        return row

    value = content_type.split(';')[0]
    row['content-type'] = value

    subfields = re.findall(r"([\w]+)\=[\"]?(.+?)[\"]?[\;|\n|$]", re.sub('[\t\s\n]+?', '', content_type))
    for (subfield, subvalue) in subfields:
        ct_subfields.add(f'content-type-{subfield.lower()}')
        row[f'content-type-{subfield}'] = subvalue
    return row


with_content_type_df = with_cc_count_df.apply(parse_content_type, axis=1)
with_content_type_df[list(ct_subfields)].describe()

,content-type-boundary,content-type-format,content-type-delsp,content-type-micalg,content-type-protocol,content-type-type,content-type-charset
count,131,13,2,180,69,39,585
unique,77,1,1,3,2,2,7
top,=_NextPart_2rfkindysadvnqw3nerasdf,flowed,yes,pgp-sha1,application/pgp-signature,multipart/alternative,us-ascii
freq,6,13,2,168,67,34,388


In [214]:
with_encoded_charset_df = one_hot_encode(with_content_type_df, 'content-type-charset')
with_encoded_charset_df[get_feat_cols(with_encoded_charset_df)]

,feat-cc-count,feat-duplicate-message-id,feat-encoded-content-transfer-encoding_7bit,feat-encoded-content-transfer-encoding_8bit,feat-encoded-content-transfer-encoding_base64,feat-encoded-content-transfer-encoding_binary,feat-encoded-content-transfer-encoding_quoted-printable,feat-encoded-precedence_bulk,feat-encoded-precedence_first-class,feat-encoded-precedence_list,...,feat-match-from-return-path,feat-match-from-sender,feat-x-msmail-priority,feat-x-priority,feat-x-spam-level,feat-x-spam-status,feat-encoded-content-type-charset_iso-8859-1,feat-encoded-content-type-charset_iso-8859-15,feat-encoded-content-type-charset_us-ascii,feat-encoded-content-type-charset_windows-1252
0,NaN,0.0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,NaN,1.0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0
10,NaN,1.0,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
15,NaN,1.0,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
20,NaN,1.0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,NaN,NaN,0,1,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0
46665,NaN,NaN,1,0,0,0,0,1,0,0,...,NaN,NaN,1.0,3.0,NaN,NaN,0,0,0,0
46675,NaN,NaN,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0
46680,NaN,NaN,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0


In [215]:
with_encoded_charset_df['feat-content-type-boundary'] = with_encoded_charset_df['content-type-boundary'].isna().astype(
    int)
with_binary_boundary_df = with_encoded_charset_df
with_binary_boundary_df[get_feat_cols(with_binary_boundary_df)]

,feat-cc-count,feat-duplicate-message-id,feat-encoded-content-transfer-encoding_7bit,feat-encoded-content-transfer-encoding_8bit,feat-encoded-content-transfer-encoding_base64,feat-encoded-content-transfer-encoding_binary,feat-encoded-content-transfer-encoding_quoted-printable,feat-encoded-precedence_bulk,feat-encoded-precedence_first-class,feat-encoded-precedence_list,...,feat-match-from-sender,feat-x-msmail-priority,feat-x-priority,feat-x-spam-level,feat-x-spam-status,feat-encoded-content-type-charset_iso-8859-1,feat-encoded-content-type-charset_iso-8859-15,feat-encoded-content-type-charset_us-ascii,feat-encoded-content-type-charset_windows-1252,feat-content-type-boundary
0,NaN,0.0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
5,NaN,1.0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1
10,NaN,1.0,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
15,NaN,1.0,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
20,NaN,1.0,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,NaN,NaN,0,1,0,0,0,1,0,0,...,NaN,NaN,NaN,0.0,0.0,0,0,0,0,1
46665,NaN,NaN,1,0,0,0,0,1,0,0,...,NaN,1.0,3.0,NaN,NaN,0,0,0,0,1
46675,NaN,NaN,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,0.0,0.0,0,0,0,0,1
46680,NaN,NaN,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,0.0,0.0,0,0,0,0,1


In [243]:
def parse_mailer(row):
    x_mailer = row['x-mailer']
    if pd.isna(x_mailer):
        return row

    result = re.sub(r"[\(\[\{].*?[\)\]\}]", "", x_mailer)
    result = re.sub(r"([\w\-]*?([\d]+\.)+[\+\d\w\-]*?)", "", result)
    result = re.sub(r"([\d\/]+)", "", result)
    result = result.lower()
    result = re.sub(r"version|with", "", result)
    result = re.sub(r"[\W]", "", result)

    row['x-mailer'] = result
    return row


with_mailer_df = with_binary_boundary_df.apply(parse_mailer, axis=1)
with_mailer_df['x-mailer'].describe()

count                        3370
unique                        152
top       microsoftoutlookexpress
freq                          652
Name: x-mailer, dtype: object

In [245]:
def bin_mailers(row, mailers):
    mailer = row['x-mailer']
    if pd.isna(mailer):
        return row

    if mailer not in mailers:
        row['x-mailer'] = 'other'

    return row


def get_top_mailers(df):
    spam_counts = df[df['category'] == 'spam']['x-mailer'].value_counts().sort_values(ascending=False)
    ham_counts = df[df['category'] == 'ham']['x-mailer'].value_counts().sort_values(ascending=False)
    ham = pd.DataFrame(ham_counts).rename(columns={'count': 'ham'})
    spam = pd.DataFrame(spam_counts).rename(columns={'count': 'spam'})
    mailer_counts = pd.concat([ham, spam], axis=1).fillna(0)
    mailer_counts['total'] = mailer_counts['spam'] + mailer_counts['ham']
    mailer_counts['spam_ratio'] = mailer_counts['spam'] / mailer_counts['total']
    return list(mailer_counts.sort_values(by=['total', 'spam_ratio'], ascending=False).iloc[:10].index)


top_mailers = get_top_mailers(with_mailer_df)
with_binned_mailer_df = with_mailer_df.apply(lambda r: bin_mailers(r, top_mailers), axis=1)
with_binned_mailer_df['x-mailer'].describe()

count      3370
unique       11
top       other
freq       1180
Name: x-mailer, dtype: object

In [246]:
with_encoded_mailer_df = one_hot_encode(with_binned_mailer_df, 'x-mailer')
with_encoded_mailer_df[get_feat_cols(with_encoded_mailer_df)]

,feat-cc-count,feat-duplicate-message-id,feat-encoded-content-transfer-encoding_7bit,feat-encoded-content-transfer-encoding_8bit,feat-encoded-content-transfer-encoding_base64,feat-encoded-content-transfer-encoding_binary,feat-encoded-content-transfer-encoding_quoted-printable,feat-encoded-precedence_bulk,feat-encoded-precedence_first-class,feat-encoded-precedence_list,...,feat-encoded-x-mailer_exmh,feat-encoded-x-mailer_internetmailservice,feat-encoded-x-mailer_microsoftcdoforwindows,feat-encoded-x-mailer_microsoftoutlookbuild,feat-encoded-x-mailer_microsoftoutlookexpress,feat-encoded-x-mailer_microsoftoutlookimobuild,feat-encoded-x-mailer_mozilla,feat-encoded-x-mailer_other,feat-encoded-x-mailer_sylpheedclaws,feat-encoded-x-mailer_ximianevolution
0,NaN,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,NaN,1.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,NaN,1.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,NaN,1.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,NaN,1.0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,NaN,NaN,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
46665,NaN,NaN,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
46675,NaN,NaN,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
46680,NaN,NaN,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
with_encoded_mailer_df['subject']

0            [Lockergnome Windows Daily]  Sticker Courtesy
5                                                      NaN
10       Great deals on perfect Summer cameras! (CNET S...
15       How Microsoft plans to take over your living r...
20            [NOVICE] pl/pgsql and returns timestamp type
                               ...                        
46660                                                  NaN
46665                                                  NaN
46675                  public mailing list sign up package
46680                                                  NaN
46685                                                  NaN
Name: subject, Length: 9154, dtype: object